#Sample Project Code
This is an incomplete project and does not contain a research hypothesis or question. It simply accessess the GDELT web API to grab the top 250 articles for a given date with keyword(s). In this case I chose three days of data for New Zealand with keyword query (coronavirus or lotto). Look at the GDELT Summary
and full text query API for more web format queries.

## setup libraries

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
!pip install newspaper3k

import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt full text query data for a given date range

Note: this example uses a query crafted using the GDELT summary

I generated this by going to https://api.gdeltproject.org/api/v2/summary/summary

I entered keyword=(coronavirus or lotto) then limited to New Zealand with a date range.
Then I scrolled down, right-clicked on the HTML link under Top Articles and pasted it below.
I modified the maxrecords=75 up to 250 which is the maximum and changed format to csv


In [3]:
queryURLbase = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=(coronavirus%20or%20lotto)%20sourcecountry:NZ%20sourcelang:eng&mode=artlist&maxrecords=250&sort=hybridrel"


In [4]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


In [5]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import os
import urllib.request

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          startdate = date+"000000" #0 hour 0 min 0 sec (start of day)
          enddate = date+"235959" #23 hour 59 min 59 sec (end of day)
          query = queryURLbase + "&startdatetime=" + startdate + "&enddatetime=" + enddate
          with urllib.request.urlopen(query) as testfile, open(filename, 'w') as f:
            f.write(testfile.read().decode())
        return filename
    except Exception as inst:
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args
        print("Error occurred")

# pull the data from gdelt into multi files; this may take a long time
daterange = [get_filename(x) for x in pd.date_range('2021 May 1','2021 May 3')]

results = list(e.map(intofile,daterange))

## read downloaded files into RDDs

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

data_urls = sqlContext.read.option("header", "true").csv(results)
dbg(data_urls)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


DataFrame[URL: string, MobileURL: string, Date: string, Title: string]


##Use the newspaper3k library from python to scrape the keywords
Maps a function over the RDD where each URL is fetched and processed by the newspaper3k library and output in a word count. This will definitely take a long time if you give it a lot of articles because it is issuing a web scrape request for each URL (the above query has 250 URLs for one day range). Then each file is parsed for keywords using an NLP engine. This is done inside a map call so if you run this with more resources (processors) it can be faster.

In [ ]:
import numpy as np
from newspaper import Article
import nltk
nltk.download('popular', quiet=True)

import time
e = ProcessPoolExecutor(16)

start_time = time.monotonic() 

def get_urls(row):
    return row['URL']
       
all_urls = data_urls.rdd.map(get_urls)
withids = all_urls.zipWithUniqueId()

if not os.path.exists('articles'):
  os.mkdir('articles')

def write_wget(withid):
  url = withid[0]
  id = withid[1]
  s = 'articles/' + str(id) + ".html "
  if not os.path.exists(os.getcwd()+'/'+s):
    print("wget " + s + url)
    os.system('wget -t 1 -T 5 -O ' + s + url)
  return id

c = withids.collect()
print(c)
results = list(e.map(write_wget, c))

end_time = time.monotonic()
print("download time took ",end_time-start_time)

start_time = time.monotonic() 

allids = withids.map(lambda x: x[1])

def output_keywords(id):
  try:
    filename = "articles/"+str(id)+".html"
    filenameurl = "file://"+os.getcwd()+'/'+filename
    article = Article(filenameurl)
    with open(filename, 'r') as f:
      s = f.read()
      article.set_html(s)
    article.parse()
    article.nlp()
    return [(k, 1) for k in article.keywords]
  except Exception as e:
    print("Exception: " + str(e))
    return []

all_keywords = allids.flatMap(
  
)
counts = all_keywords.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False)
dbg(counts)
end_time = time.monotonic()
print("processing took ",end_time-start_time)
  


[('https://www.stuff.co.nz/national/125014519/hokonui-rnanga-suggests-link-between-p-users-money-laundering-and-organised-crime-in-gore', 0), ('https://www.newshub.co.nz/home/world/2021/05/coronavirus-latest-on-covid-19-from-around-the-world-tuesday-may-4.html', 2), ('https://www.nzherald.co.nz/world/covid-19-coronavirus-india-records-highest-daily-death-toll-since-pandemic-began/2FRGHRGQTSVKGVWIDHVK23UCTM/', 4), ('https://www.stuff.co.nz/national/health/coronavirus/300291387/covid19-why-indias-pandemic-data-is-vastly-undercounted', 6), ('https://www.nzherald.co.nz/travel/eu-bloc-prepares-to-open-external-borders-to-international-visitors/ML3TVDGYTOL4UTDOFHVRFIXM7A/', 8), ('https://www.stuff.co.nz/sport/cricket/300291554/covid19-two-players-at-brendon-mccullums-kolkata-knight-riders-test-positive', 10), ('https://www.stuff.co.nz/sport/cricket/300291554/covid19-indian-premier-league-in-disarray-after-two-kolkata-knight-riders-players-test-positive', 12), ('https://www.stuff.co.nz/world/

In [ ]:
!zip -o articles.zip articles/*